### Generating the MnAu2 cell structure inputs for the QE calculations

Read in an initial .cif file (6-atoms) and expand to 54 atoms cell by translating in the x,y directions to increase the number of Mn atoms. This allows for more magnetic patterns in the Mn layers. Save to file as a .cif. 
The patterns matrix and coordinateis saved to file are used to generate the input for calculating configuation energies.
Total configuration energies and the pattern matrix are input for the exchange model to calculate the exchange parameters. 


In [232]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import itertools
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

kB = 1000 * Boltzmann / electron_volt

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

### Starting with 6 atom cell:

In [233]:
cif_filepath = "data/mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


### Getting the neighbor information to calculate the exchange parameters:

Magnetic structure read in from the 24 atom Mn8Au16 structure with determined unique (collinear) patterns.

In [234]:
cif_filepath1 = "data/mn8au16_collinear.cif"
cif_filepath2 = "data/mn8au16_1x1x4.cif"
structure_mnau2_c = from_file(structure_file=cif_filepath1)
structure_mnau2_z = from_file(structure_file=cif_filepath2)
#print(structure_mnau2_c)
print(structure_mnau2_z)

Full Formula (Mn8 Au16)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839  35.034520
angles:  90.000000  90.000000  90.000000
Sites (24)
  #  SP      a    b        c
---  ----  ---  ---  -------
  0  Mn    0.5  0.5  0.875
  1  Au    0    0    0.91492
  2  Au    0    0    0.83508
  3  Mn    0    0    0.75
  4  Au    0.5  0.5  0.71008
  5  Au    0.5  0.5  0.78992
  6  Mn    0.5  0.5  0.625
  7  Au    0    0    0.58508
  8  Au    0    0    0.66492
  9  Mn    0    0    0.5
 10  Au    0.5  0.5  0.46008
 11  Au    0.5  0.5  0.53992
 12  Mn    0.5  0.5  0.375
 13  Au    0    0    0.33508
 14  Au    0    0    0.41492
 15  Mn    0    0    0.25
 16  Au    0.5  0.5  0.21008
 17  Au    0.5  0.5  0.28992
 18  Mn    0    0    0
 19  Au    0.5  0.5  0.96008
 20  Au    0.5  0.5  0.03992
 21  Mn    0.5  0.5  0.125
 22  Au    0    0    0.08508
 23  Au    0    0    0.16492


/home/ssilayi/anaconda3/lib/python3.6/site-packages/pymatgen/core/lattice.py:694: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(q.T, p.T)[0].T


Read in and rearrange the pattern matrices to match the atomic sequence in the structure and create pattern matrix useable 
in the exchange model calculation.  

In [235]:
mn_layer1=pd.read_csv('data/mg_collinear_patterns', header=None)
au_layer = pd.DataFrame(np.zeros((2,18)))
collinear_patterns = pd.concat([mn_layer1[0:1],au_layer,
                                mn_layer1[1:2],au_layer,
                                mn_layer1[2:3],au_layer,
                                mn_layer1[3:4],au_layer,
                                mn_layer1[4:5],au_layer,
                                mn_layer1[5:6],au_layer,
                                mn_layer1[6:7],au_layer,
                                mn_layer1[7:8],au_layer], ignore_index=True)
c_types =[]
c_types.append("nm")
index = np.arange(1,len(collinear_patterns.columns))
for i in index:
    c_types.append("c-"+str(i))
    
collinear_patterns.columns = c_types

mn_layer2=pd.read_csv('data/mg_1x1x4_patterns', header=None)
au_layer = pd.DataFrame(np.zeros((2,19)))
z_patterns = pd.concat([mn_layer2[0:1],au_layer,
                                mn_layer2[1:2],au_layer,
                                mn_layer2[2:3],au_layer,
                                mn_layer2[3:4],au_layer,
                                mn_layer2[4:5],au_layer,
                                mn_layer2[5:6],au_layer,
                                mn_layer2[6:7],au_layer,
                                mn_layer2[7:8],au_layer], ignore_index=True)

z_types =[]
z_types.append("nm")
index = np.arange(2,len(z_patterns.columns)+1)
for i in index:
    z_types.append("z-"+str(i))
    
z_patterns.columns = z_types

#collinear_patterns
#z_patterns

### Calculate the neighbor energies from the magnetic patterns:

In [236]:

mnau2_neighbor_data_c = count_neighbors(cell_structure=structure_mnau2_c, r=8.8)
exchange_model_c = build_model(magnetic_patterns=collinear_patterns, neighbor_data=mnau2_neighbor_data_c)
#drop the Au atoms
exchange_model_c = exchange_model_c[exchange_model_c.columns.drop(list(exchange_model_c.filter(regex='Au')))]
#exchange_model_c

mnau2_neighbor_data_z = count_neighbors(cell_structure=structure_mnau2_z, r=8.8)
exchange_model_z = build_model(magnetic_patterns=z_patterns, neighbor_data=mnau2_neighbor_data_z)
#drop the Au atoms
exchange_model_z = exchange_model_z[exchange_model_z.columns.drop(list(exchange_model_z.filter(regex='Au')))]
#exchange_model_z

### Calculated energies for different patterns using QE:

In [237]:
#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies_c = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies_c.columns = ["pattern", "num_sites", "total_energy" ]

qe_energies_z = pd.read_csv('data/energy.out.1x1x4.csv', header=None)
qe_energies_z.columns = ["pattern", "num_sites", "total_energy" ]
qe_energies_z

,pattern,num_sites,total_energy
0,nm,24,-14691.879405
1,z-2,24,-14694.011502
2,z-3,24,-14694.008085
3,z-4,24,-14694.009522
4,z-5,24,-14694.002601
5,z-6,24,-14694.009322
6,z-7,24,-14694.005007
7,z-8,24,-14694.004323
8,z-9,24,-14694.008779
9,z-10,24,-14694.005441


### Checking magnetic moments and total energies:

Pre-process: Check that magnetic moment is not lost and that energy values are unique for the different patterns 
within a tolerance of 0.1 meV

In [238]:
#read in magnetic moments and return any that don't have the expected total magnetic moment (18)
mg_moments_c = pd.read_csv('data/mg_moment.collinear',sep='\t', header=None)
mg_moments_c = round (abs(mg_moments_c/4.0)).sum(axis=0)
mg_moments_c = pd.concat([pd.DataFrame({'pattern':c_types}), pd.DataFrame({'total_moment':mg_moments_c})], axis=1)
#mg_moments
mg_moments_c[(mg_moments_c['total_moment'] != 8 ) & (mg_moments_c['pattern'] != "nm") ] #.head()

####
####
mg_moments_z = pd.read_csv('data/mg_moment.1x1x4',sep='\t', header=None)
mg_moments_z = round (abs(mg_moments_z/4.0)).sum(axis=0)
mg_moments_z = pd.concat([pd.DataFrame({'pattern':z_types}), pd.DataFrame({'total_moment':mg_moments_z})], axis=1)
#mg_moments
mg_moments_z[(mg_moments_z['total_moment'] != 8 ) & (mg_moments_z['pattern'] != "nm") ] #.head()

,pattern,total_moment


In [247]:
#Check that all energies are not within the tolerance value of each other
def drop_duplicates(qe_energies):
    ryd_to_ev = 13.605693009 #eV 
    tol = 0.0001 #eV

    #convert to eV
    qe_energies["total_energy"] = qe_energies["total_energy"]*ryd_to_ev

    for i in range(len(qe_energies)-1):
        for j in range(1+1,len(qe_energies)):
            if (i != j):
                diff = abs(qe_energies["total_energy"][i] - qe_energies["total_energy"][j])
                if (  diff  <= tol ):
                    qe_energies["pattern"][j] = qe_energies["pattern"][i]
                    #print (i,j, qe_energies["total_energy"][i], qe_energies["total_energy"][j], diff)
                
                
    #drop repeating energies            
    qe_energies = qe_energies.reset_index(drop=True).drop_duplicates(["pattern"], keep="first")           
 
    #convert back to Rydberg
    qe_energies["total_energy"] = qe_energies["total_energy"]/ryd_to_ev
    
    return qe_energies

qe_energies1 = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies1.columns = ["pattern", "num_sites", "total_energy" ]
qe_energies_c = drop_duplicates(qe_energies1) 

qe_energies2 = pd.read_csv('data/energy.out.1x1x4.csv', header=None)
qe_energies2.columns = ["pattern", "num_sites", "total_energy" ]
qe_energies_z = drop_duplicates(qe_energies2)    

#qe_energies_c
#qe_energies_z

qe_energies_all = pd.concat([qe_energies_c, qe_energies_z])
qe_energies_all


/home/ssilayi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,pattern,num_sites,total_energy
0,nm,24,-14691.879405
1,c-1,24,-14693.951620
2,c-2,24,-14693.957454
3,c-3,24,-14693.970500
4,c-4,24,-14693.947722
5,c-5,24,-14693.946375
6,c-6,24,-14693.953194
7,c-7,24,-14693.949463
8,c-8,24,-14693.941122
9,c-9,24,-14693.932390


Calculated energies and values from neighbor model matched according to pattern type. 
The energies are normalized relative to the "nm" type (spin 0).

In [248]:
def ModelMatrix(qe_energies):
    model_matrix = qe_energies \
        .assign(energy = lambda x:
            (x["total_energy"] -
             np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
            1000 * units.Ha_to_eV / x["num_sites"])  \
        .merge(exchange_model, on=["pattern"]) \
        .query("pattern != 'nm'") \
        .assign(J1 = lambda x: x["J1_MnMn"]) \
        .assign(J2 = lambda x: x["J2_MnMn"]) \
        .assign(J3 = lambda x: x["J3_MnMn"]) \
        .assign(J4 = lambda x: x["J4_MnMn"]) \
        .assign(J5 = lambda x: x["J5_MnMn"]) \
        .assign(J6 = lambda x: x["J6_MnMn"]) \
        .assign(J7 = lambda x: x["J7_MnMn"]) \
        .loc[:, ["pattern", "energy", "J1", "J2", "J3", "J4", "J5" , "J6", "J7"]]

    return model_matrix

model_matrix = ModelMatrix(qe_energies_all)

model_matrix


ValueError: operands could not be broadcast together with shapes (36,) (2,) 

### Fitting to get final parameters:

Final matrix of energies and exchange model is used in a fit function to get the values of the J parameters.
2 results are compared, and values are in units of Rydberg.

In [161]:
smf_exchange_fit = smf.ols(data=model_matrix, formula="energy ~ J1 + J2 + J3 + J4").fit()
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
#smf_exchange_parameters

In [162]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=model_matrix[["J1", "J2", "J3", "J4"]], y=model_matrix["energy"])
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],lm_exchange_fit.coef_[3]]},
    index=["Intercept", "J1", "J2", "J3", "J4"],
)
#lm_exchange_parameters

In [163]:
exchange_parameters = pd.concat([smf_exchange_parameters,lm_exchange_parameters], axis=1)
#save to file    
exchange_parameters.to_csv("data/mnau2_exchange_parameters.collinear.txt", sep=' ')
    

exchange_parameters

,stats_models,sklearn
Intercept,-2343.262191,-2343.262191
J1,-17.678685,-17.678685
J2,-8.688744,-8.688744
J3,-0.658163,-0.658163
J4,-12.271814,-12.271814
